In [ ]:
id_num = 0000
id_num

In [ ]:
def lpad(i, width, fillchar='0'):
    """입력된 숫자 또는 문자열 왼쪽에 fillchar 문자로 패딩"""
    return str(i).rjust(width, fillchar)

In [ ]:
from main import PororoOcr
import cv2
import numpy as np
import matplotlib.pyplot as plt
# 최초에 한번은 Model을 다운로드 합니다. 경로는 Windows의 경우 ```C:/pororo/misc```이고 Mac OS의 경우는 ```~/.pororo/misc/``` 입니다.
ocr = PororoOcr()

# 현재 지원 가능하는 언어는 영어와 한국어 입니다.
ocr.get_available_langs()
ocr.get_available_models()

# 사용할 이미지
img_path = 'car77.jpg'
# 차 이미지 글자 인식
ocr.run_ocr(img_path, debug=True)

In [ ]:
# 번호판 글자 인식

longest_idx, longest_text = -1, 0
plate_chars = []

for i in range(len(ocr.get_ocr_result()['bounding_poly'])):
    result_chars = ''
    has_digit = False

    chars = ocr.get_ocr_result()['bounding_poly'][i]['description']
    for c in chars:
        if ord('가') <= ord(c) <= ord('힣') or c.isdigit():
            if c.isdigit():
                has_digit = True
            result_chars += c
    
    print(result_chars)
    plate_chars.append(result_chars)

    if has_digit and len(result_chars) > longest_text:
        longest_idx = i
        x = ocr.get_ocr_result()['bounding_poly'][longest_idx]['vertices'][0]['x']
        y = ocr.get_ocr_result()['bounding_poly'][longest_idx]['vertices'][0]['y']
        w = ocr.get_ocr_result()['bounding_poly'][longest_idx]['vertices'][1]['x'] - x
        h = ocr.get_ocr_result()['bounding_poly'][longest_idx]['vertices'][2]['y'] - y
        id_num = int(id_num) + 1
        id_num = lpad(id_num, 4, '0')
        ID = 'AA_' + id_num

In [ ]:
result_chars

In [ ]:
ocr.get_ocr_result()

print(x,y,w,h)

img_ori = cv2.imread(img_path)

In [ ]:
# 번호판 이미지 크롭
cropped_img = img_ori[y: y + h, x: x + w]

In [ ]:
# 번호판 이미지 확인
plt.figure(figsize=(12, 10))
plt.imshow(cropped_img,cmap='gray')

In [ ]:
# 번호판 이미지 저장
from PIL import Image
pil_Image = Image.fromarray(cropped_img)
pil_Image.save(f"./data/plate_{img_path}")

In [ ]:
# 번호판 이미지 글자 인식
label = ocr.run_ocr(f"./data/plate_{img_path}", debug=True)[0]

In [ ]:
# 글자 정제
plate_label = ''
for c in label:
        if ord('가') <= ord(c) <= ord('힣') or c.isdigit():
            if c.isdigit():
                has_digit = True
            plate_label += c

In [ ]:
plate_label

In [ ]:
# 오늘 날짜 저장
from datetime import datetime

today = datetime.today().strftime("%Y-%m-%d")
print(today)

In [ ]:
# json 파일 만들기
import json
from collections import OrderedDict
plate_data = OrderedDict()

try:
    with open('./data.json', 'r') as f:
       plate_data = json.load(f)
except FileNotFoundError:
    plate_data = {'data' : []}


my_dict = {"ID":ID,
           "CName":"김지수",
           "date_create":today,
           "data_path":f"./data/{result_chars}.jpg",
           "filename":img_path,
           "x":x,
           "y":y,
           "w":w,
           "h":h,
           "label":plate_label}

plate_data['data'].append(my_dict)
with open('./data.json','w') as f:
  json.dump(plate_data, f, ensure_ascii=False, indent=4)